implementing Decission Tree algorithm with the gini metric

In [4]:
import numpy as np
from collections import Counter

In [5]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        """Builds the decision tree from the training data."""
        self.tree = self._grow_tree(X, y, depth=0)

    def predict(self, X):
        """Predicts class labels for input samples."""
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _gini(self, y):
        """Computes the Gini impurity of a set of labels."""
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _best_split(self, X, y):
        """Finds the best feature and threshold to split the data."""
        best_gini = float('inf')
        best_split = None
        n_features = X.shape[1]

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                # Compute weighted Gini impurity for the split
                gini_left = self._gini(y[left_mask])
                gini_right = self._gini(y[right_mask])
                gini = (gini_left * np.sum(left_mask) + gini_right * np.sum(right_mask)) / len(y)

                if gini < best_gini:
                    best_gini = gini
                    best_split = (feature, threshold)

        return best_split

    def _grow_tree(self, X, y, depth):
        """Recursively grows the decision tree."""
        if len(set(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            return Counter(y).most_common(1)[0][0]

        split = self._best_split(X, y)
        if split is None:
            return Counter(y).most_common(1)[0][0]

        feature, threshold = split
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Recursively grow left and right subtrees
        left_subtree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        return (feature, threshold, left_subtree, right_subtree)

    def _traverse_tree(self, x, node):
        """Traverses the tree to make a prediction for a single sample."""
        if not isinstance(node, tuple):
            return node
        feature, threshold, left, right = node
        return self._traverse_tree(x, left if x[feature] <= threshold else right)

Let´s test the algorithm

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [7]:
# Ejemplo de uso
if __name__ == "__main__":
    # Sample dataset
    X = np.array([[2, 3], [1, 1], [3, 2], [5, 3], [6, 2], [7, 3], [4, 2], [8, 3]])
    y = np.array([0, 0, 0, 1, 1, 1, 1, 0])

    tree = DecisionTree(max_depth=3)
    tree.fit(X, y)
    predictions = tree.predict(X)

    # Compute evaluation metrics
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)

    print("Predicciones:", predictions)
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

Predicciones: [0 0 0 1 1 1 1 0]
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
